In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [22]:
embed = nn.Embedding(97, 10)
inputs = torch.tensor([1, 2])
print(embed.weight)
print(embed(inputs).view((1, -1)))

Parameter containing:
tensor([[-5.5813e-01, -3.8069e-01, -8.6228e-01,  7.7004e-01,  3.9818e-01,
         -6.4771e-01, -7.8602e-01, -1.1205e-01, -1.8084e+00, -4.5820e-01],
        [-1.5205e+00,  3.3611e-01,  4.2861e-01,  4.3552e-01,  1.6431e+00,
          1.9720e-01,  2.9650e-01,  1.1167e+00, -6.3198e-01, -2.0910e-01],
        [-7.1412e-01, -6.9486e-01,  8.0708e-01, -8.6596e-01, -1.4493e+00,
          1.1988e+00, -2.0179e+00,  7.2909e-01,  4.8399e-02,  2.4731e+00],
        [ 7.3870e-01,  3.6973e-01,  6.4809e-01,  3.9970e-01,  5.7724e-01,
         -4.2937e-02, -5.1354e-01, -1.1415e+00,  1.8658e+00,  1.5361e+00],
        [ 1.1934e+00, -6.7499e-01,  1.1644e+00, -3.8294e-01, -3.0984e-01,
          1.0704e+00,  9.3139e-01,  7.1325e-01, -4.8688e-03,  7.2546e-01],
        [ 1.2006e+00,  1.2302e+00,  1.0295e+00,  1.6964e-01,  1.4802e+00,
         -6.4397e-01, -1.4225e-01,  7.5939e-01,  1.7779e+00, -8.8204e-01],
        [-1.9081e+00,  3.2797e-01,  1.1902e+00,  7.9144e-01, -6.0379e-01,
         -

In [30]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128) # this simply means 2* 10 = 20, same as view() function
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# generating the word vectors of the word 'beauty'
print(model.embeddings.weight[word_to_ix['beauty']])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[522.6618957519531, 520.2036204338074, 517.7615690231323, 515.335289478302, 512.9245121479034, 510.52873516082764, 508.1458203792572, 505.776846408844, 503.4204030036926, 501.0751271247864]
tensor([ 0.3274, -0.8197, -0.2992,  0.9405,  1.1669,  0.2578,  0.6064, -0.8329,
         0.1583,  1.4086], grad_fn=<SelectBackward0>)


#### Considering both left and right window

###### Here we consider a sliding window of 5 and created a word embedding for the tokens of 10 dimensions

In [72]:
CONTEXT_SIZE = 4 # this implies a sliding window of 5
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)

ngrams = []
for i in range(CONTEXT_SIZE+1, len(test_sentence)):
    val = test_sentence[i-5:i]
    target = val[2]
    del(val[2])
    ngrams.append((val, target))
    
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])



vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128) # this simply means 4* 10 = 40, same as view() function
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# generating the word vectors of the word 'beauty'
print(model.embeddings.weight[word_to_ix['beauty']])

[(['When', 'forty', 'shall', 'besiege'], 'winters'), (['forty', 'winters', 'besiege', 'thy'], 'shall'), (['winters', 'shall', 'thy', 'brow,'], 'besiege')]
[505.26292610168457, 502.55792760849, 499.86606645584106, 497.18921518325806, 494.5260081291199, 491.87620306015015, 489.2391517162323, 486.61101746559143, 483.9915041923523, 481.3826153278351]
tensor([ 0.1978, -0.9526, -1.2305, -1.5631,  1.3720,  1.0216, -0.6120, -1.8942,
        -1.1994,  1.2345], grad_fn=<SelectBackward0>)
